## Coursera Capstone Project

### The Battle of Neighborhoods (Week 1): Where to open Pizza shop in Music city, Nashville-TN?

### Anuj Tripathi Part 1 : Introduction and Data Sections

In [1]:
import numpy as np # library to handle data in a vectorized manner
from bs4 import BeautifulSoup
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [2]:
# A description of the problem and a discussion of the background. (15 marks)

### 1. Introduction Section :

#### Discussion of the business problem and the audience who would be interested in this project

#### project: Where to open Pizza shop in Music city, Nashville-TN?

#### Introduction

Nashville is the capital of the U.S. state of Tennessee and called as Music City due to the epicenter of country and western music in the United States.  Nashville is perhaps best known as the capital of country music, as evidenced by such attractions as the Country Music Hall of Fame and the city's famous Music Row district and the legendary country music venues Grand Ole Opry House.

Today, Nashville-Tn not only music city, it is a leader in health care industry with more than 500 health care companies, 
operated from Nashville. Nashville health care industry contributes an overall economic benefit of $46.7 billion and more
than 270,000 jobs to the local economy annually.

The Nashville Convention & Visitors Bureau reports that the tourism industry broke another record, with 16.1 million 
visitors to the city in 2019. Throughout the whole year, tourism spending in Nashville topped more than $7 billion. 
Due to these specialty, now Nashville was recently named one of 8 cities with booming entrepreneur communities, mostly 
due to music scene and healthcare industry ties.

#### Business Problem 

This report focusses on the issue of where to open a new pizza store in a neighborhood of Music city. The Nashville and its neighborhoods are target the food business. More than 100 new restaurants and bars opened last year and an additional 100 will open this year. Downtown is famous for music, bars and restaurant, but neighborhoods also famous as historical spots in Music City tourists with low business competition.

##### Interested Audience

Metropolitan Government of Nashville & Davidson County invite and promotes new vendors/ entrepreneur for business opportunities in Music city. People also spent a lot of money on Nashville's food scene. We have analyzed neighborhoods of Music City and pizza store in different famous neighborhoods, to decide best and less competitive locations for our client’s new pizza shop.

In [5]:
# A description of the data and how it will be used to solve the problem. (15 marks)

### 2. Data Section:

For this project we need the following data:

A.	Nashville, TN neighborhood data that contains list Boroughs, Neighborhoods along with their Postal code. 

Data source: PostalCode_Nashville_Tn.xlsx

Description: This data set contains the required information. And we will use this data set to explore various neighborhoods of new york city. Previously scraped Collect the Nashville,TN neighborhood data  from Wikipedia (https://en.wikipedia.org/wiki/Nashville,_Tennessee) and make .xlsx file.

B.	GeoSpace data for latitude and longitude of US Zip code.

Data source: us-zip-code-latitude-and-longitude.xlsx

Description: US Zip Code Latitude and Longitude open data (https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/export/)

C.	 Id of the Pizza Place: 4bf58dd8d48988d1ca941735 

D.	Pizza stores in each neighborhood of Nashville, TN.

Data source: Fousquare API

Description: By using this api we will get all the venues in each neighborhood. We can filter these venues to get only Pizza stores.


### 3. Approach:

•	Collect the Nashville,TN neighborhood data  from Wikipedia (https://en.wikipedia.org/wiki/Nashville,_Tennessee) and make .xlsx file.

•	Load data by .xlsx to pandas dataframe.

•	For GeoSpace data, US Zip Code Latitude and Longitude.xlsx data file used

(https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/export/).

•	Find geographical coordinate of neighborhood of Nashville, TN.

•	Create map of Nashville,TN with folium and mark neighborhoods.

•	Using FourSquare API we will find pizza stores for each neighborhood.

•	Filter out pizza stores for each neighborhood.

•	Visualize pizza stores for each neighborhood .

•	Find less no pizza store neighborhood  to open a new pizza shop.


In [9]:
df_Nashville = pd.read_excel (r'C:\Users\Dr Anuj Tripathi\Desktop\PostalCode_Nashville_Tn.xlsx')
df_Nashville.head()

,PostalCode,Borough,Neighborhood
0,37013,Nashville,Antioch
1,37205,Belle Meade,Belle Meade Plantation
2,37221,Nashville,Bellevue
3,37207,Williamson,Brentwood
4,37206,Nashville,East Nashville


In [10]:
df_Nashville.shape

(27, 3)

In [12]:
coordinates_df = pd.read_excel (r'C:\Users\Dr Anuj Tripathi\Desktop\us-zip-code-latitude-and-longitude.xlsx')
coordinates_df.head()

,PostalCode,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,71937,Cove,AR,34.398483,-94.39398,-6,1,"34.398483, -94.39398"
1,72044,Edgemont,AR,35.624351,-92.16056,-6,1,"35.624351, -92.16056"
2,56171,Sherburn,MN,43.660847,-94.74357,-6,1,"43.660847, -94.74357"
3,49430,Lamont,MI,43.010337,-85.89754,-5,1,"43.010337, -85.89754"
4,52585,Richland,IA,41.194129,-91.98027,-6,1,"41.194129, -91.98027"


In [14]:
CLIENT_ID = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX' # your Foursquare ID
CLIENT_SECRET = 'LXKJXIJISIJXIJJMMMMMMMMMMMMMXXXXXXXXXXX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')

Your credentails:


In [15]:
# type your answer here
LIMIT = 500 # Maximum is 100
neighbors = ['Belle Meade, TN', 'Brentwood, TN', 'Franklin, TN', 'Goodlettsville, TN', 'Mount Juliet, TN', 'Murfreesboro, TN', 'Nashville, TN', 'Spring Hill, TN', 'Smyrna, TN']  
results = {}
for neighbor in neighbors:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        neighbor,
        LIMIT,
        "4bf58dd8d48988d1ca941735") # PIZZA PLACE CATEGORY ID
    results = requests.get(url).json()

In [16]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
venues = results['response']['groups'][0]['items']
Pizza_shop = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
Pizza_shop =Pizza_shop.loc[:, filtered_columns]

# filter the category for each row
Pizza_shop['venue.categories'] = Pizza_shop.apply(get_category_type, axis=1)

# clean columns
Pizza_shop.columns = [col.split(".")[-1] for col in Pizza_shop.columns]

Pizza_shop.head()

,name,categories,lat,lng
0,Salvo's Pizza,Pizza Place,35.979291,-86.558411
1,La Tavola Ristorante Italiano,Pizza Place,35.981155,-86.519264
2,Papa Murphy's,Pizza Place,35.985543,-86.552327
3,Pizza Hut,Pizza Place,35.993390,-86.527729
4,Jet's Pizza,Pizza Place,35.941974,-86.528232


### 4. Methodology:

### 5. Result:

### 6. Discussion:

### 7. Conclusion: